In [1]:
#Import all the required libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import sys 
import os
import tensorflow as tf
%matplotlib inline

from sklearn.model_selection import train_test_split 
from sklearn.metrics import classification_report 
from sklearn import datasets 
from tensorflow import keras
from tensorflow.keras import models, layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D, MaxPool2D
from tensorflow.python.keras.models import Sequential
from tensorflow.keras.models import Sequential
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.xception import Xception, preprocess_input
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical # convert to one-hot-encoding
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras import backend as K
from keras.layers import BatchNormalization

import warnings
warnings.filterwarnings("ignore")


In [2]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_data = train_datagen.flow_from_directory(
        'skinDisease Datasets',
        target_size=(32, 32),
        batch_size=8,
)

test_data = test_datagen.flow_from_directory(
        'skinDisease Datasets',
        target_size=(32, 32),
        batch_size=8,
        )


Found 4518 images belonging to 4 classes.
Found 4518 images belonging to 4 classes.


In [3]:
tf.keras.preprocessing.image.DirectoryIterator(
    "skinDisease Datasets",
    ImageDataGenerator,
    target_size=(256, 256),
    color_mode='rgb',
    classes=None,
    class_mode='binary',
    batch_size=32,
    shuffle=True,
    seed=None,
    data_format=None,
    save_to_dir=None,
    save_prefix='',
    save_format='png',
    follow_links=False,
    subset=None,
    interpolation='nearest',
    keep_aspect_ratio=False,
    dtype=None
)

Found 4518 images belonging to 4 classes.


In [4]:
#Set all the Constants
batchSize = 32
imageSize = 256
channels= 3
epoch = 70 

In [5]:
#Import data into tensorflow dataset object()
imageDataset = tf.keras.preprocessing.image_dataset_from_directory(
    "skinDisease Datasets", seed=123, shuffle=True, image_size=(imageSize,imageSize), batch_size=batchSize
)

Found 4518 files belonging to 4 classes.


In [6]:
#Check for dataset classes
classNames = imageDataset.class_names
classNames

['Acne', 'Eczema', 'Melanoma Skin Cancer Nevi and Moles', 'Psoriasis']

In [7]:
pre_trained_model = Xception(input_shape=(256, 256, 3), include_top=False, weights="imagenet")

In [8]:
for layer in pre_trained_model.layers:
    print(layer.name)
    layer.trainable = False
    
print(len(pre_trained_model.layers))

input_1
block1_conv1
block1_conv1_bn
block1_conv1_act
block1_conv2
block1_conv2_bn
block1_conv2_act
block2_sepconv1
block2_sepconv1_bn
block2_sepconv2_act
block2_sepconv2
block2_sepconv2_bn
conv2d
block2_pool
batch_normalization
add
block3_sepconv1_act
block3_sepconv1
block3_sepconv1_bn
block3_sepconv2_act
block3_sepconv2
block3_sepconv2_bn
conv2d_1
block3_pool
batch_normalization_1
add_1
block4_sepconv1_act
block4_sepconv1
block4_sepconv1_bn
block4_sepconv2_act
block4_sepconv2
block4_sepconv2_bn
conv2d_2
block4_pool
batch_normalization_2
add_2
block5_sepconv1_act
block5_sepconv1
block5_sepconv1_bn
block5_sepconv2_act
block5_sepconv2
block5_sepconv2_bn
block5_sepconv3_act
block5_sepconv3
block5_sepconv3_bn
add_3
block6_sepconv1_act
block6_sepconv1
block6_sepconv1_bn
block6_sepconv2_act
block6_sepconv2
block6_sepconv2_bn
block6_sepconv3_act
block6_sepconv3
block6_sepconv3_bn
add_4
block7_sepconv1_act
block7_sepconv1
block7_sepconv1_bn
block7_sepconv2_act
block7_sepconv2
block7_sepconv2_

In [9]:
last_layer = pre_trained_model.get_layer('block14_sepconv2_act')
print('last layer output shape:', last_layer.output_shape)
last_output = last_layer.output

last layer output shape: (None, 8, 8, 2048)


In [10]:
# Flatten the output layer to 1 dimension
x = layers.GlobalMaxPooling2D()(last_output)
# Add a fully connected layer with 512 hidden units and ReLU activation
x = layers.Dense(512, activation='relu')(x)
# Add a dropout rate of 0.5
x = layers.Dropout(0.5)(x)
# Add a final sigmoid layer for classification
x = layers.Dense(1)(x)

# Configure and compile the model
model = Model(pre_trained_model.input, x)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [11]:
model.fit(
        train_data,
        validation_data = train_data,
        verbose=1,
        epochs=70,
       )

Epoch 1/70
 15/565 [..............................] - ETA: 2:10 - loss: 0.6859 - accuracy: 0.7375

KeyboardInterrupt: 

In [ ]:
for layer in pre_trained_model.layers:
    layer.trainable = True

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', patience=3, verbose=1, factor=0.5, 
                                            min_lr=0.000001, cooldown=2)

In [ ]:
model.summary()

In [ ]:
loss_val, acc_val = model.evaluate(train_data, verbose=1)
print("Validation: accuracy = %f  ;  loss_v = %f" % (acc_val, loss_val))

In [ ]:
loss_test, acc_test = model.evaluate(test_data, verbose=1)
print("Test: accuracy = %f  ;  loss = %f" % (acc_test, loss_test))

In [ ]:
model_version=max([int(i) for i in os.listdir("../skinDiseaseDetection/models") + [2]])
model.save(f"../skinDiseaseDetection/models/{model_version}")

In [ ]:
model.save("../modelXception.h5")